# Autoregressive (AR) and Auto Regressive Integrated Moving Average (ARIMA) Models

**Introduction:**

A statistical model is autoregressive if it predicts future values based on past values. For example, an autoregressive model might seek to predict a stock's future prices based on its past performance [1].An AR(1) autoregressive process is one in which the current value is based on the immediately preceding value, while an AR(2) process is one in which the current value is based on the previous two values. An AR(0) process is used for white noise and has no dependence between the terms [1]

In [1]:
# this code is used to upload dataset from Pc to colab
from google.colab import files # Please First run this cod in chrom 
def getLocalFiles():
    _files = files.upload() # upload StudentNextSessionf.csv datase
    if len(_files) >0: # Then run above  libray 
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving daily-min-temperatures.csv to daily-min-temperatures.csv


## 1 - Importing necessary libraries ##
we import the following python packges for this tutorial 
 - pandas: importing index data
 - matplotlib: it is used for ploting data
 - statsmodels build time series models
 - statistic to help find statistic of models

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  
from statsmodels.tsa.arima_model import ARIMA 
import statistics

In [5]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

In [6]:
from pandas import Series
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error

## 2 - Load Datasets ##

In [ ]:
daily_temp = pd.read_csv('daily-min-temperatures.csv', usecols=['Date','Temp'],index_col='Date', parse_dates=True )
daily_temp.head()

## 3- Data Description # 

In [ ]:
daily_temp.index

In [ ]:
daily_temp.head()

In [ ]:

daily_temp['1981-01-01':'1981-01-05']

## 3- Check Null value

In [ ]:
daily_temp.isna().sum()

Temp    0
dtype: int64

## 4 - Ploting ##


In [ ]:
 daily_temp.plot(df['Temp'])

In [ ]:
%matplotlib inline
daily_temp.Temp.resample("M").mean().plot()

In [ ]:
daily_temp.Temp.resample("W").mean().plot()

In [ ]:
daily_temp.Temp.resample("D").mean().plot()

In [ ]:
Quarterly_resampled_data = daily_temp.Temp.resample('Q').mean().plot()

In [ ]:
x=pd.Series(range(3), index=pd.date_range('1981', freq='D', periods=3))

In [ ]:
x

In [ ]:
mx=pd.Series(range(3), index=pd.date_range('2019', freq='D', periods=3))


In [ ]:
daily_temp.Temp.resample('QS').mean().plot()

In [ ]:
daily_temp.Temp.resample('M').mean().plot(kind="line")

##5- Correlation ##
Visual check of correlation by plotting the scatter plot against the previous and next time step Pandas built in plot - Lagplot - to do visual check



**Stationary**

To check wither data is stationaly or not. stationary mean that is data is constant in mean and variance . we cehck this becasue many time sieries model required that data should be stationary in order to model it. It should be concentatn movment up and down momement

In [ ]:
plt.plot(daily_temp)
plt.show()

now we make data stationary if data is not stationary 
  - Difference the data and make it satationary . difference mean substract the next value 
  - fill the miss value data

In [9]:
# take serveral diff untill your data become stationary 
daily_temp_diff1=daily_temp.diff().fillna(daily_temp)

In [ ]:
plt.plot(daily_temp)
plt.show()

**Autocorrelation Function (ACF) plot**

**Partial Autocorrelation Function(PACF) Plot ** 

Helps determine number of autoregressive AR term and moving average term

can also be used to help see seasonality or periodic trends


In [ ]:
plot_acf(daily_temp)
plt.show()

In [ ]:
plot_pacf(daily_temp)
plt.show()

## 6-Data Spliting ##

In [13]:
 X = daily_temp.values
train, test = X[1:len(X)-7], X[len(X)-7:]

##6 -Autoregressive model##
Forecast the next timestamp value by regressing over previous vaues

### 6.1 - Models training on training Data ###

```
# This is formatted as code
```




In [14]:
from statsmodels.tsa.ar_model import AR

In [15]:
model = AR(train)
model_fit = model.fit()

In [ ]:
print('Lag: %s' % model_fit.k_ar)
print('Coefficients: %s' % model_fit.params)

### 6.2 - Model prediction on test data### 

In [17]:
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)

In [ ]:
for i in range(len(predictions)):
    print('predicted=%f, expected=%f' % (predictions[i], test[i]))

### 7.3 Model Evaulation###

In [19]:
error = mean_squared_error(test, predictions)

In [ ]:
print('Test MSE: %.3f' % error)

In [ ]:
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

### 6.4 -Model tunning

In [22]:
window = model_fit.k_ar
coef = model_fit.params

In [23]:
history = train[len(train)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()

In [ ]:
for t in range(len(test)):
    length = len(history)
    lag = [history[i] for i in range(length-window,length)]
    yhat = coef[0]
    for d in range(window):
        yhat += coef[d+1] * lag[window-d-1]
    obs = test[t]
    predictions.append(yhat)
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

### 6.5 - Model Evaulation ###

In [ ]:
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

In [ ]:
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

## 7 - Auto Regressive Integrated Moving Average (ARIMA) ##

 Forecast the next timestamps value by averaging over the previous values 
  - useful for non-stationary data
  - allows us to difference the data
  - also includes a seasona differencing parameter for seasonal non-stationary data 

### 7.1 Models training on training Data

In [29]:
model_ARIMA = ARIMA(train, order=(5,0,1))
model_fit_ARIMA = model_ARIMA.fit(disp=0)

In [ ]:
print('Lag: %s' % model_fit_ARIMA.k_ar)
print('Coefficients: %s' % model_fit_ARIMA.params)

### 7.2 - Model prediction on test data. ###


In [31]:
predictions_ARIMA = model_fit_ARIMA.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)

In [ ]:
for i in range(len(predictions_ARIMA)):
    print('predicted=%f, expected=%f' % (predictions_ARIMA[i], test[i]))

In [ ]:
# Another Mothod 
from statsmodels.tsa.arima_model import ARMA
mod = ARMA(train, order=(0,1))
res = mod.fit()
res.plot_predict(start=1990, end=1991)
plt.show()
predictions_ARIMA = res.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
for i in range(len(predictions_ARIMA)):
    print('predicted=%f, expected=%f' % (predictions_ARIMA[i], test[i]))


###7.3 Model Evaulation##

In [ ]:
error_ARIMA = mean_squared_error(test, predictions_ARIMA)

In [ ]:
print('Test MSE: %.3f' % error_ARIMA)

In [ ]:
plt.plot(test)
plt.plot(predictions_ARIMA, color='red')
plt.show()

###7.4 Model tunning##

In [ ]:
window_ARIMA = model_fit_ARIMA.k_ar
coef_ARIMA = model_fit_ARIMA.params

In [ ]:
history_ARIMA = train[len(train)-window_ARIMA:]
history_ARIMA = [history_ARIMA[i] for i in range(len(history_ARIMA))]
predictions_ARIMA = list()

In [ ]:
for t in range(len(test)):
    length = len(history_ARIMA)
    lag = [history_ARIMA[i] for i in range(length-window,length)]
    yhat = coef[0]
    for d in range(window_ARIMA):
        yhat += coef[d+1] * lag[window_ARIMA-d-1]
    obs = test[t]
    predictions_ARIMA.append(yhat)
    history_ARIMA.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

###7.5 Model Evaulation##

References:

Autoregressive¶
https://www.investopedia.com/terms/a/autoregressive.asp

Autoregressive Models
https://online.stat.psu.edu/stat501/lesson/14/14.1#:~:text=An%20autoregressive%20model%20is%20when,from%20that%20same%20time%20series.&text=The%20order%20of%20an%20autoregression,value%20at%20the%20present%20time.

ARIMA modeling and forecasting: Time Series in Python Part 2
https://tutorials.datasciencedojo.com/arima-model-time-series-python/